# construct keras model and load pretrained weight 

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import load_model
import numpy as np

from keras.wrappers.scikit_learn import KerasClassifier

# build function for the Keras' scikit-learn API
def create_keras_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.load_weights('nhwc_model.h5')
    model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    return model

/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# load datasets for testing purpose 

In [2]:
from keras.datasets import mnist
# input image dimensions
img_rows, img_cols = 28, 28
num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# wrap keras model into Scikit_learn API 

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier ,Sequential,absolute_import

classifier = KerasClassifier(build_fn=create_keras_model,nb_epoch=1)

classifier.fit(x_train,y_train, epochs=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
60000/60000 [==============================] - 29s 479us/step - loss: 0.0357 - acc: 0.9892


### ensure the scikit-learn Keras API loaded model can indeed make prediction correctly and successfully 

In [4]:
import random
import numpy as np
rn=random.randint(0,len(x_train))
sample_x=x_train[rn].reshape(1,28,28,1)
out=classifier.predict(sample_x)
print(" predicted result is :{} , actual label is :{}".format(str(out[0]),str(np.argmax(y_train[rn]))))

 predicted result is :5 , actual label is :5


# load TensorRT as inference engine and wrap scikit-learn keras API model into it's inference engine

In [3]:
import keras.backend as K
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
import tensorflow as tf
import tensorrt as trt

import time
import uff
import utils.ascii as helper
import utils.dataset as data
from pdb import set_trace
from tensorflow.contrib import tensorrt as tftrt

import copy
import numpy as np
import sys
import time

import utils.ascii as helper
import utils.dataset as data
img_h = x_test.shape[1]
img_w = x_test.shape[2]
#helper.print_ascii(x_test[0], img_h, img_w)

In [4]:
class FrozenGraph(object):
    def __init__(self, model, shape):
        shape = (None, shape[0], shape[1], shape[2])
        x_name = 'image_tensor_x'
        with K.get_session() as sess:
            x_tensor = tf.placeholder(tf.float32, shape, x_name)
            K.set_learning_phase(0)
            y_tensor = model(x_tensor)
            y_name = y_tensor.name[:-2]
            graph = sess.graph.as_graph_def()
            graph0 = tf.graph_util.convert_variables_to_constants(sess, graph, [y_name])
            graph1 = tf.graph_util.remove_training_nodes(graph0)

        self.x_name = [x_name]
        self.y_name = [y_name]
        self.frozen = graph1

class TfEngine(object):
    def __init__(self, graph):
        g = tf.Graph()
        with g.as_default():
            x_op, y_op = tf.import_graph_def(
              graph_def=graph.frozen, return_elements=graph.x_name + graph.y_name)
            self.x_tensor = x_op.outputs[0]
            self.y_tensor = y_op.outputs[0]

            config = tf.ConfigProto(gpu_options=
            tf.GPUOptions(per_process_gpu_memory_fraction=0.5,
            allow_growth=True))

        self.sess = tf.Session(graph=g, config=config)

    def infer(self, x):
        y = self.sess.run(self.y_tensor,
          feed_dict={self.x_tensor: x})
        return y

class TftrtEngine(TfEngine):
    def __init__(self, graph, batch_size, precision):
        tftrt_graph = tftrt.create_inference_graph(
          graph.frozen,
          outputs=graph.y_name,
          max_batch_size=batch_size,
          max_workspace_size_bytes=1 << 30,
          precision_mode=precision,
          minimum_segment_size=2)

        opt_graph = copy.deepcopy(graph)
        opt_graph.frozen = tftrt_graph
        super(TftrtEngine, self).__init__(opt_graph)
        self.batch_size = batch_size

    def infer(self, x):
        num_tests = x.shape[0]
        y = np.empty((num_tests, self.y_tensor.shape[1]), np.float32)
        batch_size = self.batch_size

        for i in range(0, num_tests, batch_size):
            x_part = x[i : i + batch_size]
            y_part = self.sess.run(self.y_tensor,
            feed_dict={self.x_tensor: x_part})
            y[i : i + batch_size] = y_part
        return y

def verify(result, ans):
    num_tests = ans.shape[0]
    error = 0
    for i in range(0, num_tests):
        a = np.argmax(ans[i])
        r = np.argmax(result[i])
        if (a != r) : error += 1

    if (error == 0) : print('PASSED')
    else            : print('FAILURE')

### convert y_test to both OHE and Numerical format for convenience

In [5]:

y_test_num=np.argmax(y_test,axis=1)
y_test_num.shape , y_test.shape

((10000,), (10000, 10))

# verify how long it takes for the scikit-learn classifier to make prediction

In [8]:
# use Keras scikit learn --> learner to do infer
t0 = time.time()
y_learner = classifier.predict(x_test)
t1 = time.time()
print('Keras time', t1 - t0)
verify(y_learner, y_test_num)

Keras time 0.7310318946838379
PASSED


### save the model back to architecture and weight to convert it back to keras 

In [9]:
#classifier.model.model.summary()

# saving model
json_model = classifier.model.to_json()
open('AL2keras.json', 'w').write(json_model)
# saving weights
classifier.model.save_weights('AL2keras.h5', overwrite=True)

### load the saved architecture and weight back to keras model 

In [6]:
# loading model
from keras.models import model_from_json

model = model_from_json(open('AL2keras.json').read())
model.load_weights('AL2keras.h5')

model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# use TensorRT to accelerate prediction with keras as input model 

In [11]:
frozen_graph = FrozenGraph(model, (img_h, img_w, 1))

tf_engine = TfEngine(frozen_graph)
t0 = time.time() 
y_tf = tf_engine.infer(x_test)
print(y_tf.shape)
t1 = time.time()
print('Tensorflow time', t1 - t0)
y_tf_c=np.argmax(y_tf,axis=1)
y_tf_c.shape
verify(y_tf_c,y_test_num)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
(10000, 10)
Tensorflow time 1.223820447921753
PASSED


In [12]:
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to /tmp/pip-req-build-u0tz7vd3
  Running command git clone -q https://github.com/modAL-python/modAL.git /tmp/pip-req-build-u0tz7vd3
  Created wheel for modAL: filename=modAL-0.3.4-cp35-none-any.whl size=25056 sha256=65dfc409305ccbd70ba86894a9a8334bd76da88bce36cda148897d512df9f5cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ozc4dao0/wheels/a6/04/97/a4f7cb2aad1616972b8f2a9bd4fb52d84fbc44f5e9245f05bf
Successfully built modAL
You should consider upgrading via the 'pip install --upgrade pip' command.


# use Active Learning to continous optimize the model's performance 

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier ,Sequential,absolute_import
import numpy as np

classifier = KerasClassifier(build_fn=create_keras_model,nb_epoch=1)
classifier.fit(x_train,y_train, epochs=1)

# assemble initial data
n_initial = 1000
initial_idx = np.random.choice(range(len(x_train)), size=n_initial, replace=False)
X_initial = x_train[initial_idx]
y_initial = y_train[initial_idx]

# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(x_train, initial_idx, axis=0)[:5000]
y_pool = np.delete(y_train, initial_idx, axis=0)[:5000]
from modAL.models import ActiveLearner

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=X_initial, y_training=y_initial,
    verbose=1
)


Instructions for updating:
Use tf.cast instead.
Epoch 1/1
60000/60000 [==============================] - 17s 280us/step - loss: 0.0340 - acc: 0.9896
Epoch 1/1
1000/1000 [==============================] - 1s 677us/step - loss: 0.0323 - acc: 0.9920


In [4]:
# the active learning loop
n_queries = 10
for idx in range(n_queries):
    print('Query no. %d' % (idx + 1))
    query_idx, query_instance = learner.query(X_pool, n_instances=100, verbose=0)
    learner.teach(
        X=X_pool[query_idx], y=y_pool[query_idx], only_new=True,
        verbose=1
    )
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)

Query no. 1
Epoch 1/1
100/100 [==============================] - 0s 5ms/step - loss: 0.6812 - acc: 0.7000
Query no. 2
Epoch 1/1
100/100 [==============================] - 1s 5ms/step - loss: 0.2807 - acc: 0.9200
Query no. 3
Epoch 1/1
100/100 [==============================] - 1s 6ms/step - loss: 0.1244 - acc: 0.9600
Query no. 4
Epoch 1/1
100/100 [==============================] - 1s 6ms/step - loss: 0.0220 - acc: 1.0000
Query no. 5
Epoch 1/1
100/100 [==============================] - 1s 7ms/step - loss: 0.0183 - acc: 1.0000
Query no. 6
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 0.0099 - acc: 1.0000
Query no. 7
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 0.0061 - acc: 1.0000
Query no. 8
Epoch 1/1
100/100 [==============================] - 1s 9ms/step - loss: 0.0069 - acc: 1.0000
Query no. 9
Epoch 1/1
100/100 [==============================] - 1s 10ms/step - loss: 0.0051 - acc: 1.0000
Query no. 10
Epoch 1/1
100/100 [=============

# check active-learned-model's performance against TensorRT accelerated model inferencing speed

In [8]:
# use Keras scikit learn --> learner to do infer
import time
t0 = time.time()
y_learner = learner.predict(x_test)
print(y_learner.shape)
t1 = time.time()
print('Keras time', t1 - t0)
#verify(y_learner, y_test_num)

(10000,)
Keras time 0.8184890747070312


In [9]:
print(y_test.shape)
y_test_num=np.argmax(y_test,axis=1)
verify(y_learner, y_test_num)

(10000, 10)
PASSED


### save the learner (active-learning fine-tuned model) and save it back to keras compatible format

In [10]:
# save the learner (active-learning fine-tuned model) and save it back to keras compatible format
learner.estimator.model.save('learner2keras.h5')
from keras.models import load_model
loaded=load_model('learner2keras.h5')
loaded.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [11]:
frozen_graph = FrozenGraph(loaded, (28, 28, 1))

tf_engine = TfEngine(frozen_graph)
t0 = time.time() 
y_tf = tf_engine.infer(x_test)
print(y_tf.shape)
t1 = time.time()
print('Tensorflow time', t1 - t0)
y_tf_c=np.argmax(y_tf,axis=1)
y_tf_c.shape
verify(y_tf_c,y_test_num)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
(10000, 10)
Tensorflow time 1.275697946548462
PASSED
